# Install Library yang diperlukan

Ini adalah script untuk mengubah pdf lampiran hasil CPNS jadi csv
Caveat: kalo langsung prosess 20rb halaman, agak keselek di tengah, jadi mending di-iterate beberapa halaman
Saya yakin ada cara yang lebih elegan. Feel free to give me any suggestions

Cara run </br>
python export_data_to_csv.py index_halaman_start index_halaman_end

Misal </br>
python export_data_to_csv.py 0 100

In [89]:
# import library
import tabula
import camelot
import pandas as pd
import datetime as dt
import sys

pd.options.display.max_columns = None

## Scraping Data

In [80]:
# Tenaga Puskemas

# Replace "nama_file.pdf" with your actual file path
df = tabula.read_pdf("2020.pdf", pages="40")[0]

df.rename(columns={'Jenis Tenaga Kesehatan': "Jenis Tenaga", "Jumlah Tenaga Kesehatan" : "Jumlah Tenaga"}, inplace = True)
df['Jumlah Tenaga'] = df['Jumlah Tenaga'].astype(str)
df['Jumlah Tenaga'] = df['Jumlah Tenaga'].str.replace(".", "")

df['Jumlah Tenaga'] = df['Jumlah Tenaga'].astype(int)
df['tahun'] = "2020"
df['unit_kerja'] = "puskesmas"
df

<ipython-input-80-ffd0afbe4c44>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Jumlah Tenaga'] = df['Jumlah Tenaga'].str.replace(".", "")


,Jenis Tenaga,Jumlah Tenaga,tahun,unit_kerja
0,Dokter atau dokter layanan primer,2784,2020,puskesmas
1,Dokter gigi,9430,2020,puskesmas
2,Perawat,10772,2020,puskesmas
3,Bidan,15616,2020,puskesmas
4,Tenaga Kefarmasian,1746,2020,puskesmas
5,Tenaga Kesmas,1411,2020,puskesmas
6,Tenaga kesehatan Lingkungan,9710,2020,puskesmas
7,Tenaga gizi,1054,2020,puskesmas
8,Ahli Teknologi Laboratorium Medik,7890,2020,puskesmas


In [21]:
# Tenaga Rumah Sakit

tables = camelot.read_pdf("2020.pdf", flavor='stream', pages='44')
df = tables[0].df
df = df.iloc[:12,: ]
df = df.iloc[3:,: ]
df.set_axis(["Jenis Tenaga Kesehatan", "Jumlah Tenaga Kesehatan"], axis=1, inplace=True)
df.rename(columns={'Jenis Tenaga Kesehatan': "Jenis Tenaga", "Jumlah Tenaga Kesehatan" : "Jumlah Tenaga"}, inplace = True)
df['Jumlah Tenaga'] = df['Jumlah Tenaga'].astype(str)
df['Jumlah Tenaga'] = df['Jumlah Tenaga'].str.replace(".", "")

df['Jumlah Tenaga'] = df['Jumlah Tenaga'].astype(int)
df['tahun'] = "2020"
df['unit_kerja'] = "rumah sakit"
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 3 to 11
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Jenis Tenaga   9 non-null      object
 1   Jumlah Tenaga  9 non-null      int32 
 2   tahun          9 non-null      object
 3   unit_kerja     9 non-null      object
dtypes: int32(1), object(3)
memory usage: 384.0+ bytes


<ipython-input-21-60a10bb5218d>:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Jumlah Tenaga'] = df['Jumlah Tenaga'].str.replace(".", "")


In [28]:
## Rekapitulasi Rasio Tenaga Kesehatan /100.000

tables = camelot.read_pdf("2020.pdf", flavor='stream', pages='45')
df = tables[0].df
df = df.iloc[2:,:]
df.set_axis(["jenis_tenaga","Ratio/100.000 Penduduk"], axis=1, inplace=True)
df['Ratio/100.000 Penduduk'] = df['Ratio/100.000 Penduduk'].str.replace(',','.')
df['Ratio/100.000 Penduduk'] = df['Ratio/100.000 Penduduk'].astype(float)
df.replace("Tenaga\rTeknisKefarmasian", "Tenaga Teknis Kefarmasian", inplace=True)


df['tahun'] = "2020"
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 2 to 13
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   jenis_tenaga            12 non-null     object 
 1   Ratio/100.000 Penduduk  12 non-null     float64
 2   tahun                   12 non-null     object 
dtypes: float64(1), object(2)
memory usage: 420.0+ bytes


<ipython-input-28-21ac6c8f5f6b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Ratio/100.000 Penduduk'] = df['Ratio/100.000 Penduduk'].str.replace(',','.')
<ipython-input-28-21ac6c8f5f6b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Ratio/100.000 Penduduk'] = df['Ratio/100.000 Penduduk'].astype(float)
<ipython-input-28-21ac6c8f5f6b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pand

In [131]:
## Jumlah Fasilitas Kesehatan Menurut Kepemilikan 
### Data ini dicek secara manual pada setiap laporan kesehatan per kab/kota 2022

tables = camelot.read_pdf("2020.pdf", flavor='stream', pages='149')

df = tables[0].df

df = df.iloc[10:,[1,30] ]
df = df.iloc[:-1,: ]

df.set_axis(["Kabupaten/Kota", "Jumlah"], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

df['Jumlah'] = df['Jumlah'].str.replace("-", "0")
df['Jumlah'] = df['Jumlah'].replace('', '0')
df['Jumlah'] = df['Jumlah'].str.replace(".", "")
df['Jumlah'] = pd.to_numeric(df['Jumlah'], errors='coerce').fillna(0).astype(int)
df['tahun'] = "2020"
df

,Kabupaten/Kota,Jumlah,tahun
0,KABUPATEN BOGOR,298,2022
1,KABUPATEN SUKABUMI,31,2022
2,KABUPATEN CIANJUR,0,2022
3,KABUPATEN BANDUNG,13,2022
4,KABUPATEN GARUT,6,2022
5,KABUPATEN TASIKMALAYA,10,2022
6,KABUPATEN CIAMIS,8,2022
7,KABUPATEN KUNINGAN,63,2022
8,KABUPATEN CIREBON,251,2022
9,KABUPATEN MAJALENGKA,51,2022


In [97]:
### PELAYANAN KESEHATAN GIGI DAN MULUT MENURUT KECAMATAN DAN PUSKESMAS

tables = camelot.read_pdf("2020.pdf", flavor='stream', pages='229')

df = tables[0].df

df = df.iloc[6:,[1,3,5,7] + list(range(9, df.shape[1])) ]
df.reset_index(drop=True, inplace=True)
df.iloc[27,0] = "Jawa Barat"

df.set_axis(["Kabupaten/Kota", "Jumlah Tumpatan Gigi Tetap", "Pencabutan Gigi Tetap", "Rasio Pencabutan", "Jumlah Kasus Gigi", "Jumlah Kasus dirujuk", "% Rasio Dirujuk"], axis=1, inplace=True)
# Fill value Kota Banjar
df.loc[df['Kabupaten/Kota'] == 'Kota Banjar', 'Jumlah Tumpatan Gigi Tetap'] = "260"
df.loc[df['Kabupaten/Kota'] == 'Kota Banjar', 'Pencabutan Gigi Tetap'] = "881"
df.loc[df['Kabupaten/Kota'] == 'Kota Banjar', 'Rasio Pencabutan'] = "0.3"
df.loc[df['Kabupaten/Kota'] == 'Kota Banjar', 'Jumlah Kasus Gigi'] = "10872"
df.loc[df['Kabupaten/Kota'] == 'Kota Banjar', 'Jumlah Kasus dirujuk'] = "534"
df.loc[df['Kabupaten/Kota'] == 'Kota Banjar', '% Rasio Dirujuk'] = "0.049"
# Fill value Kota Tasik
df.loc[df['Kabupaten/Kota'] == 'Kota Tasikmalaya', 'Jumlah Kasus Gigi'] = "2225"

# Fill value Kab Ciamis
df.loc[df['Kabupaten/Kota'] == 'Kabupaten Ciamis', 'Jumlah Tumpatan Gigi Tetap'] = "559"
df.loc[df['Kabupaten/Kota'] == 'Kabupaten Ciamis', 'Pencabutan Gigi Tetap'] = "402"
df.loc[df['Kabupaten/Kota'] == 'Kabupaten Ciamis', 'Rasio Pencabutan'] = "1.5"


df = df.replace('', '0')
df = df.replace('-', '0')
df = df.replace('#DIV/0!', '0')
for col in ['Rasio Pencabutan', '% Rasio Dirujuk']:
    df[col] = df[col].str.replace(',', '.').astype(float)

for col in ['Jumlah Tumpatan Gigi Tetap', 'Pencabutan Gigi Tetap', 'Jumlah Kasus Gigi', 'Jumlah Kasus dirujuk']:
    df[col] = df[col].str.replace('.', '').astype(int)
df['tahun'] = "2020"
df['unit_kerja'] = "puskesmas"
df

<ipython-input-97-c67a200c2c3e>:35: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('.', '').astype(int)


,Kabupaten/Kota,Jumlah Tumpatan Gigi Tetap,Pencabutan Gigi Tetap,Rasio Pencabutan,Jumlah Kasus Gigi,Jumlah Kasus dirujuk,% Rasio Dirujuk,tahun,unit_kerja
0,Kabupaten Bogor,6285,9828,0.6,133537,3428,2.600,2020,puskesmas
1,Kabupaten Sukabumi,0,0,0.0,0,0,0.000,2020,puskesmas
2,Kabupaten Cianjur,496,232,2.1,0,0,0.000,2020,puskesmas
3,Kabupaten Bandung,8608,13040,0.7,126778,4244,3.300,2020,puskesmas
4,Kabupaten Garut,1238,2757,0.4,10050,286,2.800,2020,puskesmas
5,Kabupaten Tasikmalaya,1385,3103,0.4,19722,308,1.600,2020,puskesmas
6,Kabupaten Ciamis,559,402,1.5,0,0,0.000,2020,puskesmas
7,Kabupaten Kuningan,459,1272,0.4,16340,1397,8.500,2020,puskesmas
8,Kabupaten Cirebon,2772,2818,1.0,81935,2864,3.500,2020,puskesmas
9,Kabupaten Majalengka,848,2801,0.3,46057,1638,3.600,2020,puskesmas


In [98]:
jawabarat_row = df[df['Kabupaten/Kota'] != 'Jawa Barat']
total_tumpatan = jawabarat_row['Jumlah Tumpatan Gigi Tetap'].sum()
total_pencabutan = jawabarat_row['Pencabutan Gigi Tetap'].sum()
total_kasus_gigi = jawabarat_row['Jumlah Kasus Gigi'].sum()
total_kasus_dirujuk = jawabarat_row['Jumlah Kasus dirujuk'].sum()

# Ganti nilai pada baris dengan Kabupaten/Kota == 'Jawa Barat'
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', 'Jumlah Tumpatan Gigi Tetap'] = total_tumpatan
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', 'Pencabutan Gigi Tetap'] = total_pencabutan
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', 'Jumlah Kasus Gigi'] = total_kasus_gigi
df.loc[df['Kabupaten/Kota'] == 'Jawa Barat', 'Jumlah Kasus dirujuk'] = total_kasus_dirujuk
df

,Kabupaten/Kota,Jumlah Tumpatan Gigi Tetap,Pencabutan Gigi Tetap,Rasio Pencabutan,Jumlah Kasus Gigi,Jumlah Kasus dirujuk,% Rasio Dirujuk,tahun,unit_kerja
0,Kabupaten Bogor,6285,9828,0.6,133537,3428,2.600,2020,puskesmas
1,Kabupaten Sukabumi,0,0,0.0,0,0,0.000,2020,puskesmas
2,Kabupaten Cianjur,496,232,2.1,0,0,0.000,2020,puskesmas
3,Kabupaten Bandung,8608,13040,0.7,126778,4244,3.300,2020,puskesmas
4,Kabupaten Garut,1238,2757,0.4,10050,286,2.800,2020,puskesmas
5,Kabupaten Tasikmalaya,1385,3103,0.4,19722,308,1.600,2020,puskesmas
6,Kabupaten Ciamis,559,402,1.5,0,0,0.000,2020,puskesmas
7,Kabupaten Kuningan,459,1272,0.4,16340,1397,8.500,2020,puskesmas
8,Kabupaten Cirebon,2772,2818,1.0,81935,2864,3.500,2020,puskesmas
9,Kabupaten Majalengka,848,2801,0.3,46057,1638,3.600,2020,puskesmas


In [104]:
## PELAYANAN KESEHATAN GIGI DAN MULUT PADA ANAK SD DAN SETINGKAT MENURUT JENIS KELAMIN, KECAMATAN, DAN PUSKESMAS

tables = camelot.read_pdf("2020.pdf", flavor='stream', pages='230')

df = tables[0].df

columns_to_drop = [18,20,22, 24, 27]
df = df.iloc[9:, [1, 2, 4, 5] + list(range(7, df.shape[1]))].drop(columns=columns_to_drop)

df = df.iloc[:-1]
df = df.iloc[:,]
df.reset_index(drop=True, inplace=True)
# df = df.rename_axis(None, axis=1)
df.iloc[27,0] = "Jawa Barat"
df.set_axis(['Kabupaten/Kota', 'jumlah_SD/MI','jumlah_SD/MI_SG_Massal','%jumlah_SD/MI_SG_Massal',
             'jumlah_SD/MI_LG','%jumlah_SD/MI_LG',
            'jumlah_murid_SD/MI_L', 'jumlah_murid_SD/MI_P', 'jumlah_murid_SD/MI_LP',
            'jumlah_murid_SD/MI_L_Diperiksa','%jumlah_murid_SD/MI_L_Diperiksa','jumlah_murid_SD/MI_P_Diperiksa','%jumlah_murid_SD/MI_P_Diperiksa', 'jumlah_murid_SD/MI_LP_Diperiksa','%jumlah_murid_SD/MI_LP_Diperiksa',
            'jumlah_murid_SD/MI_L_PR','jumlah_murid_SD/MI_P_PR','jumlah_murid_SD/MI_LP_PR',
            'jumlah_murid_SD/MI_L_MP','%jumlah_murid_SD/MI_L_MP','jumlah_murid_SD/MI_P_MP','%jumlah_murid_SD/MI_P_MP','jumlah_murid_SD/MI_LP_MP','%jumlah_murid_SD/MI_LP_MP'], axis=1, inplace=True)
df = df.replace('', '0')
df = df.replace('-', '0')
# Kota Banjar
additional_values_str = ['111', '86', '77.48', '94', '84.68', '10,099', '9,242', '19,341', '1,672', '16.56', '1,629', '17.63', '3,301', '17.07', '0', '0', '0', '0', '0']
additional_values_str = [value.replace(',', '') for value in additional_values_str]
df.loc[df['Kabupaten/Kota'] == 'Kota Banjar', df.columns[1:len(additional_values_str)+1]] = additional_values_str


## Replace #DIV N #Value
for col in df.columns:
    df.loc[df[col].astype(str).str.contains('#'), col] = 0

for col in df.columns[df.columns.str.contains('%')]:
    df[col] = df[col].str.replace(',', '.').astype(float)

for col in df.columns[~(df.columns.str.contains('%') | (df.columns == 'Kabupaten/Kota'))]:
    df[col] = df[col].replace('\.', '', regex=True).astype(int)
df.fillna(0, inplace=True)

tahun = "2020"
df['tahun'] = tahun
df.to_csv(f"dataset/pelayanan_gigi_anaksd/{tahun}.csv", index=False)
df

,Kabupaten/Kota,jumlah_SD/MI,jumlah_SD/MI_SG_Massal,%jumlah_SD/MI_SG_Massal,jumlah_SD/MI_LG,%jumlah_SD/MI_LG,jumlah_murid_SD/MI_L,jumlah_murid_SD/MI_P,jumlah_murid_SD/MI_LP,jumlah_murid_SD/MI_L_Diperiksa,%jumlah_murid_SD/MI_L_Diperiksa,jumlah_murid_SD/MI_P_Diperiksa,%jumlah_murid_SD/MI_P_Diperiksa,jumlah_murid_SD/MI_LP_Diperiksa,%jumlah_murid_SD/MI_LP_Diperiksa,jumlah_murid_SD/MI_L_PR,jumlah_murid_SD/MI_P_PR,jumlah_murid_SD/MI_LP_PR,jumlah_murid_SD/MI_L_MP,%jumlah_murid_SD/MI_L_MP,jumlah_murid_SD/MI_P_MP,%jumlah_murid_SD/MI_P_MP,jumlah_murid_SD/MI_LP_MP,%jumlah_murid_SD/MI_LP_MP,tahun
0,Kabupaten Bogor,1684,1684,100.00,1684,100.00,125595,120192,245787,2508,2.00,2409,2.00,4917,2.00,1592,1485,3077,904,56.8,816,54.9,1720,55.9,2020
1,Kabupaten Sukabumi,1297,450,34.70,414,31.90,85364,86154,171518,427,0.50,1253,1.50,1680,1.00,594,642,1236,134,22.6,197,30.7,331,26.8,2020
2,Kabupaten Cianjur,1188,79,6.60,0,0.00,0,0,0,0,0.00,0,0.00,0,0.00,0,0,0,0,0.0,0,0.0,0,0.0,2020
3,Kabupaten Bandung,3066,1184,38.60,1255,40.90,136695,127208,263903,25832,18.90,19019,15.00,44851,17.00,9395,8990,18385,2934,31.2,3001,33.4,5935,32.3,2020
4,Kabupaten Garut,1721,0,0.00,450,26.10,78591,77373,155964,3834,4.90,3968,5.10,7802,5.00,2123,2091,4214,1045,49.2,1154,55.2,2199,52.2,2020
5,Kabupaten Tasikmalaya,847,286,33.80,689,133.30,3261,3906,7167,27253,835.70,26266,672.50,53519,0.00,7510,7859,15369,1688,22.5,2144,27.3,3832,24.9,2020
6,Kabupaten Ciamis,0,0,0.00,0,0.00,0,0,0,0,0.00,0,0.00,0,0.00,0,0,0,0,0.0,0,0.0,0,0.0,2020
7,Kabupaten Kuningan,566,67,11.80,0,0.00,37808,34096,71904,3669,9.70,3615,10.60,7284,10.10,518,541,1059,276,53.3,279,51.6,555,52.4,2020
8,Kabupaten Cirebon,1070,0,0.00,1058,98.90,0,0,111423,0,0.00,0,0.00,35068,31.50,0,0,7500,0,0.0,0,0.0,2936,39.1,2020
9,Kabupaten Majalengka,714,0,0.00,604,56.40,0,0,0,0,0.00,0,0.00,0,0.00,0,0,0,0,0.0,0,0.0,0,0.0,2020


In [82]:
## Jumlah Tenaga Medis Per Kab/Kota Puskesmas
tables = camelot.read_pdf("2020.pdf", flavor='stream', pages='182')

df = tables[1].df

df = df.iloc[:32, [1] + [20,22,24,26,28,30]]

df = df.iloc[5: ]
df.set_axis(['Kab/Kota','jml_dg_l','jml_dg_p','jml_dg_lp','jml_dgsp_l','jml_dgsp_p','jml_dgsp_lp'], axis=1, inplace=True)
df = df.replace('', '0')
df = df.replace('-', '0')

for col in df.columns[~(df.columns.str.contains('%') | (df.columns == 'Kab/Kota'))]:
    df[col] = df[col].str.replace(',', '').astype(int)
df

,Kab/Kota,jml_dg_l,jml_dg_p,jml_dg_lp,jml_dgsp_l,jml_dgsp_p,jml_dgsp_lp
5,Kabupaten Bogor,5,56,61,0,0,0
6,Kabupaten Sukabumi,9,14,23,0,0,0
7,Kabupaten Cianjur,15,51,66,0,0,0
8,Kabupaten Bandung,16,62,78,0,0,0
9,Kabupaten Garut,7,16,23,0,0,0
10,Kabupaten Tasikmalaya,9,15,24,0,0,0
11,Kabupaten Ciamis,3,13,16,0,0,0
12,Kabupaten Kuningan,5,11,16,0,0,0
13,Kabupaten Cirebon,9,31,40,0,0,0
14,Kabupaten Majalengka,3,11,14,0,0,0


In [83]:
# Menggabungkan kolom 'jml_dg_l', 'jml_dg_p', 'jml_dgsp_l', dan 'jml_dgsp_p' menjadi satu kolom 'jumlah'
df_dg = pd.concat([
    df.loc[:,['Kab/Kota','jml_dg_l']].rename(columns={'jml_dg_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi', unit_kerja='puskesmas'),
    df.loc[:,['Kab/Kota','jml_dg_p']].rename(columns={'jml_dg_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi', unit_kerja='puskesmas'),
    df.loc[:,['Kab/Kota','jml_dgsp_l']].rename(columns={'jml_dgsp_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi spesialis', unit_kerja='puskesmas'),
    df.loc[:,['Kab/Kota','jml_dgsp_p']].rename(columns={'jml_dgsp_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi spesialis', unit_kerja='puskesmas')
], ignore_index=True)

# Menghapus duplikat dan mengurutkan data berdasarkan 'Kab/Kota' dan 'jns_kelamin'
df_dg_pks = df_dg.drop_duplicates().sort_values(['Kab/Kota','jns_kelamin'])

In [84]:
## Jumlah Tenaga Medis Kab/Kota Rumah Sakit

tables = camelot.read_pdf("2020.pdf", flavor='stream', pages='182')

df = tables[0].df

df = df.iloc[33:, [1] + [20,22,24,26,28,30]]

df.set_axis(['Kab/Kota','jml_dg_l','jml_dg_p','jml_dg_lp','jml_dgsp_l','jml_dgsp_p','jml_dgsp_lp'], axis=1, inplace=True)
df = df.replace('', '0')
df = df.replace('-', '0')

for col in df.columns[~(df.columns.str.contains('%') | (df.columns == 'Kab/Kota'))]:
    df[col] = df[col].str.replace(',', '').astype(int)

In [85]:
tables = camelot.read_pdf("2020.pdf", flavor='stream', pages='183')

df2 = tables[0].df

df2 = df2.iloc[3:, [1] + [18,20,22,24,26,28]]
df2.set_axis(['Kab/Kota','jml_dg_l','jml_dg_p','jml_dg_lp','jml_dgsp_l','jml_dgsp_p','jml_dgsp_lp'], axis=1, inplace=True)
df2 = df2.replace('', '0')
df2 = df2.replace('-', '0')

for col in df2.columns[~(df2.columns.str.contains('%') | (df2.columns == 'Kab/Kota'))]:
    df2[col] = df2[col].str.replace(',', '').astype(int)


df = pd.concat([df, df2], ignore_index=True)

In [86]:
df_dg = pd.concat([
    df.loc[:,['Kab/Kota','jml_dg_l']].rename(columns={'jml_dg_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi', unit_kerja='rumah sakit'),
    df.loc[:,['Kab/Kota','jml_dg_p']].rename(columns={'jml_dg_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi', unit_kerja='rumah sakit'),
    df.loc[:,['Kab/Kota','jml_dgsp_l']].rename(columns={'jml_dgsp_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi spesialis', unit_kerja='rumah sakit'),
    df.loc[:,['Kab/Kota','jml_dgsp_p']].rename(columns={'jml_dgsp_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi spesialis', unit_kerja='rumah sakit')
], ignore_index=True)

# Menghapus duplikat dan mengurutkan data berdasarkan 'Kab/Kota' dan 'jns_kelamin'
df_dg_rs = df_dg.drop_duplicates().sort_values(['Kab/Kota','jns_kelamin'])

In [87]:
# ## Klinik DIKLAT KESEHATAN
df_sukabumi = pd.DataFrame({
    'Kab/Kota': ['Kota Sukabumi'],
    'jml_dg_l': [3],
    'jml_dg_p': [14],
    'jml_dgsp_l': [0],
    'jml_dgsp_p': [0]
})

df_bekasi = pd.DataFrame({
    'Kab/Kota': ['Kabupaten Bekasi'],
    'jml_dg_l': [31],
    'jml_dg_p': [139],
    'jml_dgsp_l': [11],
    'jml_dgsp_p': [28]
})

df_cimahi = pd.DataFrame({
    'Kab/Kota': ['Kota Cimahi'],
    'jml_dg_l': [16],
    'jml_dg_p': [21],
    'jml_dgsp_l': [1],
    'jml_dgsp_p': [0]
})

df_combined = pd.concat([df_sukabumi, df_bekasi,df_cimahi], ignore_index=True)

df_dg = pd.concat([
    df_combined.loc[:,['Kab/Kota','jml_dg_l']].rename(columns={'jml_dg_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi', unit_kerja='klinik diklat kesehatan'),
    df_combined.loc[:,['Kab/Kota','jml_dg_p']].rename(columns={'jml_dg_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi', unit_kerja='klinik diklat kesehatan'),
    df_combined.loc[:,['Kab/Kota','jml_dgsp_l']].rename(columns={'jml_dgsp_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi spesialis', unit_kerja='klinik diklat kesehatan'),
    df_combined.loc[:,['Kab/Kota','jml_dgsp_p']].rename(columns={'jml_dgsp_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi spesialis', unit_kerja='klinik diklat kesehatan')
], ignore_index=True)

# Menghapus duplikat dan mengurutkan data berdasarkan 'Kab/Kota' dan 'jns_kelamin'
df_dg_dikkes = df_dg.drop_duplicates().sort_values(['Kab/Kota','jns_kelamin'])

In [88]:
## Klinik Dinas Kesehatan
data_sukabumi = {'Kab/Kota': 'Kota Sukabumi',
              'jml_dg_l': 3,
              'jml_dg_p': 14,
              'jml_dgsp_l': 0,
              'jml_dgsp_p': 0}

data_bandung = {'Kab/Kota': 'Kota Bandung',
                'jml_dg_l': 4,
                'jml_dg_p': 16,
                'jml_dgsp_l': 0,
                'jml_dgsp_p': 0}

data_tasik = {'Kab/Kota': 'Kota Tasikmalaya',
                'jml_dg_l': 24,
                'jml_dg_p': 44,
                'jml_dgsp_l': 18,
                'jml_dgsp_p': 20}

df_sukabumi = pd.DataFrame([data_sukabumi])
df_bandung = pd.DataFrame([data_bandung])
df_tasik = pd.DataFrame([data_tasik])

df_combined = pd.concat([df_sukabumi, df_bandung, df_tasik], ignore_index=True)

df_dg = pd.concat([
    df_combined.loc[:,['Kab/Kota','jml_dg_l']].rename(columns={'jml_dg_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi', unit_kerja='klinik DINKES'),
    df_combined.loc[:,['Kab/Kota','jml_dg_p']].rename(columns={'jml_dg_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi', unit_kerja='klinik DINKES'),
    df_combined.loc[:,['Kab/Kota','jml_dgsp_l']].rename(columns={'jml_dgsp_l':'jumlah'}).assign(jns_kelamin='laki-laki', profesi='dokter gigi spesialis', unit_kerja='klinik DINKES'),
    df_combined.loc[:,['Kab/Kota','jml_dgsp_p']].rename(columns={'jml_dgsp_p':'jumlah'}).assign(jns_kelamin='perempuan', profesi='dokter gigi spesialis', unit_kerja='klinik DINKES')
], ignore_index=True)

# Menghapus duplikat dan mengurutkan data berdasarkan 'Kab/Kota' dan 'jns_kelamin'
df_dg_dinkes = df_dg.drop_duplicates().sort_values(['Kab/Kota','jns_kelamin'])
df_dg = pd.concat([df_dg_pks,df_dg_rs,df_dg_dikkes,df_dg_dinkes], ignore_index=True)
tahun = "2020"
df_dg['tahun'] = tahun
df_dg.to_csv(f'dataset/dg_unit_kerja/{tahun}.csv', index=False)
df_dg

,Kab/Kota,jumlah,jns_kelamin,profesi,unit_kerja,tahun
0,Kabupaten Bandung,16,laki-laki,dokter gigi,puskesmas,2020
1,Kabupaten Bandung,0,laki-laki,dokter gigi spesialis,puskesmas,2020
2,Kabupaten Bandung,62,perempuan,dokter gigi,puskesmas,2020
3,Kabupaten Bandung,0,perempuan,dokter gigi spesialis,puskesmas,2020
4,Kabupaten Bandung Barat,7,laki-laki,dokter gigi,puskesmas,2020
...,...,...,...,...,...,...
235,Kota Sukabumi,0,perempuan,dokter gigi spesialis,klinik DINKES,2020
236,Kota Tasikmalaya,24,laki-laki,dokter gigi,klinik DINKES,2020
237,Kota Tasikmalaya,18,laki-laki,dokter gigi spesialis,klinik DINKES,2020
238,Kota Tasikmalaya,44,perempuan,dokter gigi,klinik DINKES,2020


In [15]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 9 to 35
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Kab/Kota     27 non-null     object
 1   jml_dg_l     27 non-null     int32 
 2   jml_dg_p     27 non-null     int32 
 3   jml_dg_lp    27 non-null     int32 
 4   jml_dgsp_l   27 non-null     int32 
 5   jml_dgsp_p   27 non-null     int32 
 6   jml_dgsp_lp  27 non-null     int32 
dtypes: int32(6), object(1)
memory usage: 996.0+ bytes
None
